# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import weather_api_key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
weather_df = pd.read_csv("../output/city_data.csv")
weather_df

,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bluff,-46.60,168.33,48.99,96,100,3.00,NZ,1601788520
1,Kapaa,22.08,-159.32,82.40,78,40,11.41,US,1601788712
2,Puerto Ayora,-0.74,-90.35,64.00,91,11,1.99,EC,1601788712
3,Okhotsk,59.38,143.30,50.72,60,28,4.25,RU,1601788712
4,Rørvik,64.86,11.24,55.00,71,16,30.91,NO,1601788713
...,...,...,...,...,...,...,...,...,...
544,Rurrenabaque,-14.44,-67.53,71.56,93,47,0.54,BO,1601788853
545,Los Llanos de Aridane,28.66,-17.92,71.60,88,75,13.87,ES,1601788853
546,Ranong,9.97,98.63,86.00,79,20,5.82,TH,1601788853
547,Atherton,53.52,-2.49,51.01,99,100,6.13,GB,1601788854


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

# Plot Heatmap
figure_layout = {
    'width': '950px',
    'height': '500px',
    'border': '1px solid black',
}
fig = gmaps.figure(layout=figure_layout, center=(0, -5), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', width='950px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Narrow down the DataFrame to find your ideal weather condition
ideal_weather_df = weather_df.loc[(weather_df["Max Temp"] >= 75) & (weather_df["Max Temp"] <= 90) &
                                  (weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0)].dropna()
ideal_weather_df

,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,Kumluca,36.37,30.29,78.33,28,0,5.59,TR,1601788521
137,Makkah al Mukarramah,21.43,39.83,86.43,42,0,5.95,SA,1601788745
147,San Patricio,19.22,-104.70,76.66,82,0,3.87,MX,1601788706
168,Petauke,-14.24,31.33,75.47,30,0,5.99,ZM,1601788752
189,Atar,20.52,-13.05,81.34,29,0,6.04,MR,1601788756
194,Abu Dhabi,24.47,54.37,86.00,70,0,3.36,AE,1601788667
277,São Miguel do Araguaia,-13.28,-50.16,81.23,25,0,3.11,BR,1601788779
282,Sawākin,19.11,37.33,84.72,68,0,4.12,SD,1601788781
351,Bukama,-9.20,25.85,76.21,63,0,1.32,CD,1601788798
371,Riyadh,24.69,46.72,77.00,36,0,3.83,SA,1601788542


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}
hotel_names = []

for index, row in ideal_weather_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and convert response to json
    hotel_info = requests.get(base_url, params=params).json()
 
    try:
        hotel_names.append(hotel_info["results"][0]["name"])  
    except:
        hotel_names.append("No hotel found")

ideal_weather_df["Hotel Name"] = hotel_names
hotel_df = ideal_weather_df.loc[ideal_weather_df["Hotel Name"] != "No hotel found"]
hotel_df

,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
49,Kumluca,36.37,30.29,78.33,28,0,5.59,TR,1601788521,TurkuazKöy
137,Makkah al Mukarramah,21.43,39.83,86.43,42,0,5.95,SA,1601788745,Pullman Zamzam Makkah
147,San Patricio,19.22,-104.70,76.66,82,0,3.87,MX,1601788706,Grand Isla Navidad Resort
168,Petauke,-14.24,31.33,75.47,30,0,5.99,ZM,1601788752,Eriboma Executive Lodge
189,Atar,20.52,-13.05,81.34,29,0,6.04,MR,1601788756,Odar kanawal
194,Abu Dhabi,24.47,54.37,86.00,70,0,3.36,AE,1601788667,Royal M Hotel & Resort Abu Dhabi
277,São Miguel do Araguaia,-13.28,-50.16,81.23,25,0,3.11,BR,1601788779,Hotel Veredas do Araguaia
282,Sawākin,19.11,37.33,84.72,68,0,4.12,SD,1601788781,Arab Plaza Hotel عرب بلازا هوتيل
371,Riyadh,24.69,46.72,77.00,36,0,3.83,SA,1601788542,InterContinental Riyadh
397,Sharjah city,25.36,55.40,87.80,70,0,5.82,AE,1601788810,"The Chedi Al Bait, Sharjah - a GHM hotel"


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [32]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', width='950px'))